<a href="https://colab.research.google.com/github/kyama/edinet_xbrl/blob/master/edinet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:

xbrlfile = '/content/jpcrp030000-asr-001_E03497-000_2018-12-31_01_2019-03-26.xbrl'

In [5]:
from lxml import etree
import json
from bs4 import BeautifulSoup
import html
import re

class XBRLparser:
    def __init__(self):
      pass

    def parse(self,xbrlfile):
        ret = {}
        ns = {
            'xbrli': 'http://www.xbrl.org/2003/instance',
            'jpcrp_cor': 'http://disclosure.edinet-fsa.go.jp/taxonomy/jpcrp/2022-11-01/jpcrp_cor'
        }

        xpaths = [None]*3
        xpaths[0] = {'bs':"//jpcrp_cor:ConsolidatedBalanceSheetTextBlock[@contextRef='CurrentYearDuration']", #貸借対照表
                  'pl':"//jpcrp_cor:ConsolidatedStatementOfComprehensiveIncomeSingleStatementTextBlock[@contextRef='CurrentYearDuration']",#連結損益及び包括利益計算書
                  'cs':"//jpcrp_cor:ConsolidatedStatementOfChangesInEquityTextBlock[@contextRef='CurrentYearDuration']",#連結株主資本等変動計算書
                  'cf':"//jpcrp_cor:ConsolidatedStatementOfCashFlowsTextBlock[@contextRef='CurrentYearDuration']"}#連結キャッシュ・フロー計算書
        xpaths[1] = {'bs':"//xbrli:xbrl/jpcrp_cor:ConsolidatedBalanceSheetTextBlock[@contextRef='CurrentYearDuration']", #貸借対照表
                  'pl':"//xbrli:xbrl/jpcrp_cor:ConsolidatedStatementOfIncomeTextBlock[@contextRef='CurrentYearDuration']",#連結損益及び包括利益計算書
                  'cs':"//xbrli:xbrl/jpcrp_cor:ConsolidatedStatementOfChangesInEquityTextBlock[@contextRef='CurrentYearDuration']",#連結株主資本等変動計算書
                  'cf':"//xbrli:xbrl/jpcrp_cor:ConsolidatedStatementOfCashFlowsTextBlock[@contextRef='CurrentYearDuration']"}#連結キャッシュ・フロー計算書
        xpaths[2] = {'bs':"//jpcrp_cor:BalanceSheetTextBlock[@contextRef='CurrentYearDuration']", #貸借対照表
                  'pl':"//jpcrp_cor:StatementOfIncomeTextBlock[@contextRef='CurrentYearDuration']",#損益及び包括利益計算書
                  'cf':"//jpcrp_cor:StatementOfCashFlowsTextBlock[@contextRef='CurrentYearDuration']"}#キャッシュ・フロー計算書
        tree = etree.parse(xbrlfile)
        root = tree.getroot()
        for xpathss in xpaths:
          try:
            for key,xpath in xpathss.items():
              escapedhtml = root.xpath(xpath,namespaces=root.nsmap)
              print(xpath)
              ret[key] = html.unescape(escapedhtml[0].text)
            break
          except IndexError as e:
            print(str(e),xpathss)
            continue
          except Exception as e:
              print('error',str(e))
        return ret

    @staticmethod
    def find_scale(unescaped_html):
        m = re.findall(r'[（(]単位：(.*)円[)）]',unescaped_html)

        scale_factor = 1
        if m[0] =='千':
          scale_factor= 1000
        elif m[0] == '百万':
          scale_factor = 1000000

        return scale_factor

    def parsebs(self,unescaped_html):
        names = ['資産の部', '負債の部', '純資産の部']
        ends = {'資産の部': '資産合計', '負債の部': '負債合計', '純資産の部': '純資産合計'}

        soup = BeautifulSoup(unescaped_html, 'html.parser')

        scale_factor = self.find_scale(unescaped_html)

        tables = soup.find_all('table')
        data = {}

        def process_section(section_name):
            section_data = {}
            current_data = section_data
            section_active = False
            indent_levels = {0: section_data}  # Resetting level mapping to a safe initial value

            for table in tables:
                rows = table.find_all('tr')
                for row in rows:
                    cells = row.find_all('td')
                    if len(cells) != 3:
                        continue

                    subject_cell, prev_year_cell, current_year_cell = cells
                    subject_text = subject_cell.text.strip().split('※')[0].strip()

                    if subject_text == section_name:
                        section_active = True
                        current_data = section_data
                        indent_levels[0] = current_data
                        continue

                    if subject_text == ends[section_name]:
                        prev_value = re.sub(r'[^0-9,-]', '', prev_year_cell.text.strip().replace('△', '-')).replace(',', '')
                        curr_value = re.sub(r'[^0-9,-]', '', current_year_cell.text.strip().replace('△', '-')).replace(',', '')
                        section_data[subject_text] = {
                            '前年': float(prev_value) * scale_factor if prev_value else None,
                            '当年': float(curr_value) * scale_factor if curr_value else None
                        }
                        section_active = False
                        continue

                    if section_active:
                        indent_style = subject_cell.get('style', '')
                        margin_left = re.search(r'margin-left: (\d+(?:\.\d+)?)px;', indent_style)
                        indent_level = float(margin_left.group(1)) if margin_left else 0

                        if indent_level in [10, 27.333333333333333, 44.666666666666666, 62]:
                            current_data = indent_levels.get(indent_level, current_data)

                        prev_value = re.sub(r'[^0-9,-]', '', prev_year_cell.text.strip().replace('△', '-')).replace(',', '')
                        curr_value = re.sub(r'[^0-9,-]', '', current_year_cell.text.strip().replace('△', '-')).replace(',', '')
                        current_data[subject_text] = {
                            '前年': float(prev_value) * scale_factor if prev_value else None,
                            '当年': float(curr_value) * scale_factor if curr_value else None
                        }
                        # Update current data at this level
                        indent_levels[indent_level] = current_data

            return section_data

        for name in names:
            data[name] = process_section(name)

        with open('bs.json', 'w', encoding='utf-8') as file:
            json.dump(data, file, indent=4, ensure_ascii=False)

        return data


    def parsepl(self,unescaped_html):
        def parse_html_to_json():
            soup = BeautifulSoup(unescaped_html, 'html.parser')

            result = {}
            current_section = None

            scale_factor = self.find_scale(unescaped_html)

            rows = soup.find_all('tr')
            for row in rows:
                cells = row.find_all('td')
                if not cells:
                    continue

                if len(cells[0].find_all('p'))==0:
                  continue
                first_cell = cells[0]
                margin_left = first_cell.find_all('p')[0].get('style', '')
                description = first_cell.find_all('p')[0].text.strip().split('※')[0].strip()  # Remove any footnote symbols and their numbers
                if 'margin-left: 10px;' in margin_left:
                    # This is a main section
                    if description not in result:
                      result[description] = {}
                    current_section = result[description]
                elif 'margin-left: 27.333333333333333333333333333px;' in margin_left and current_section is not None:
                    # This is a subsection of the current section
                    if description not in current_section:
                        current_section[description] = {}
                if current_section is None:
                    continue
                if len(cells) == 3:
                    cell1 = cells[1].text.strip().replace('△','-')
                    cell2 = cells[2].text.strip().replace('△','-')
                    m1 = re.sub(r'[^0-9,-]','',cell1)
                    m2 = re.sub(r'[^0-9,-]','',cell2)
                    # Columns for previous and current year values
                    prior_year = m1.replace(',','')
                    prior_year = 0 if prior_year == '' else int(prior_year) * scale_factor
                    current_year = m2.replace(',','')
                    current_year = 0 if current_year == '' else int(current_year) * scale_factor
                    # Update or create new entry in the current section
                    if description in current_section:
                        # Avoid duplicating keys by ensuring the subsection is correctly placed
                        current_section[description].update({"前年": prior_year, "当年": current_year})
                    else:
                        current_section[description] = {"前年": prior_year, "当年": current_year}

            # Save the result to a JSON file
            with open('pl.json', 'w', encoding='utf-8') as json_file:
                json.dump(result, json_file, ensure_ascii=False, indent=4)
            return result

        parsed_data = parse_html_to_json()
        print(json.dumps(parsed_data, indent=4, ensure_ascii=False))

    def parsecf(self,unescaped_html):
        def parse_html_to_json():
            soup = BeautifulSoup(unescaped_html, 'html.parser')

            result = {}
            current_section = None


            scale_factor = self.find_scale(unescaped_html)
            rows = soup.find_all('tr')
            for row in rows:
                cells = row.find_all('td')
                if not cells:
                    continue
                if len(cells[0].find_all('p'))==0:
                    continue
                first_cell = cells[0]
                margin_left = first_cell.find_all('p')[0].get('style', '')
                description = first_cell.find_all('p')[0].text.strip().split('※')[0].strip()  # Remove any footnote symbols and their numbers
                if 'margin-left: 10px;' in margin_left:
                    # This is a main section
                    if description not in result:
                      result[description] = {}
                    current_section = result[description]
                elif 'margin-left: 27.333333333333333333333333333px;' in margin_left and current_section is not None:
                    # This is a subsection of the current section
                    if description not in current_section:
                        current_section[description] = {}
                if current_section is None:
                    continue
                if len(cells) == 3:
                    cell1 = cells[1].text.strip().replace('△','-')
                    cell2 = cells[2].text.strip().replace('△','-')
                    m1 = re.sub(r'[^0-9,-]','',cell1)
                    m2 = re.sub(r'[^0-9,-]','',cell2)
                    # Columns for previous and current year values
                    prior_year = m1.replace(',','')
                    prior_year = 0 if prior_year == '-' else prior_year
                    prior_year = 0 if prior_year == '' else int(prior_year) * scale_factor
                    current_year = m2.replace(',','')
                    current_year = 0 if current_year == '-' else current_year
                    current_year = 0 if current_year == '' else int(current_year) * scale_factor
                    # Update or create new entry in the current section
                    if description in current_section:
                        # Avoid duplicating keys by ensuring the subsection is correctly placed
                        current_section[description].update({"前年": prior_year, "当年": current_year})
                    else:
                        current_section[description] = {"前年": prior_year, "当年": current_year}

            # Save the result to a JSON file
            with open('cf.json', 'w', encoding='utf-8') as json_file:
                json.dump(result, json_file, ensure_ascii=False, indent=4)
            return result

        parsed_data = parse_html_to_json()
        print(json.dumps(parsed_data, indent=4, ensure_ascii=False))




In [6]:
#import XBRLParser

xbrlp = XBRLparser()
unescaped_html = xbrlp.parse(xbrlfile)


with open('bs.html','w') as file:
  file.write(unescaped_html['bs'])
with open('pl.html','w') as file:
  file.write(unescaped_html['pl'])
with open('cf.html','w') as file:
  file.write(unescaped_html['cf'])

bs_data = xbrlp.parsebs(unescaped_html['bs'])
pl_data = xbrlp.parsepl(unescaped_html['pl'])
cf_data = xbrlp.parsecf(unescaped_html['cf'])

//jpcrp_cor:ConsolidatedBalanceSheetTextBlock[@contextRef='CurrentYearDuration']
//jpcrp_cor:ConsolidatedStatementOfComprehensiveIncomeSingleStatementTextBlock[@contextRef='CurrentYearDuration']
//jpcrp_cor:ConsolidatedStatementOfChangesInEquityTextBlock[@contextRef='CurrentYearDuration']
//jpcrp_cor:ConsolidatedStatementOfCashFlowsTextBlock[@contextRef='CurrentYearDuration']
{
    "売上高": {
        "売上高": {
            "前年": 88347986000,
            "当年": 109553023000
        }
    },
    "売上原価": {
        "売上原価": {
            "前年": 61418845000,
            "当年": 77525768000
        }
    },
    "売上総利益": {
        "売上総利益": {
            "前年": 26929140000,
            "当年": 32027255000
        }
    },
    "販売費及び一般管理費": {
        "販売費及び一般管理費": {
            "前年": 15091418000,
            "当年": 18236593000
        }
    },
    "営業利益": {
        "営業利益": {
            "前年": 11837722000,
            "当年": 13790661000
        }
    },
    "営業外収益": {
        "営業外収益": {
            "前年": 0,
 

In [7]:
def get_value(data, keys):
    """
    Helper function to retrieve values from JSON data based on a list of possible keys.
    """
    for key in keys:
        if key in data:
            return data[key]
    return 0

In [8]:

def calculate_roic(bs_data, pl_data):
    # 税引後営業利益の計算
    operating_profit_keys = ['営業利益']
    operating_profit = get_value(pl_data['営業利益'], operating_profit_keys)['当年']

    tax_keys = ['法人税等合計']
    pre_tax_profit_keys = ['税金等調整前当期純利益']
    tax_amount = get_value(pl_data['法人税等合計'], tax_keys)['当年']
    pre_tax_profit = get_value(pl_data['税金等調整前当期純利益'], pre_tax_profit_keys)['当年']
    tax_rate = tax_amount / pre_tax_profit if pre_tax_profit != 0 else 0

    nopat = operating_profit * (1 - tax_rate)

    # 投下資本の計算
    total_assets_keys = ['資産合計']
    current_liabilities_keys = ['流動負債合計']
    total_assets = get_value(bs_data['資産の部'], total_assets_keys)['当年']
    current_liabilities = get_value(bs_data['負債の部'], current_liabilities_keys)['当年']
    invested_capital = total_assets - current_liabilities

    # ROICの計算
    roic = nopat / invested_capital if invested_capital != 0 else 0

    return roic

# JSONファイルからデータを読み込む
with open('bs.json', 'r', encoding='utf-8') as f:
    bs_data = json.load(f)

with open('pl.json', 'r', encoding='utf-8') as f:
    pl_data = json.load(f)

# ROICの計算
roic = calculate_roic(bs_data, pl_data)

print(f"ROIC: {roic:.2%}")


ROIC: 28.65%


In [9]:

def calculate_wacc(bs_data, pl_data):
    # 株主資本コストの計算
    risk_free_rate = 0.01  # 安全利子率（例として1%を使用）
    market_risk_premium = 0.06  # 市場リスクプレミアム（例として6%を使用）
    beta = 1.2  # ベータ値（例として1.2を使用）
    cost_of_equity = risk_free_rate + beta * market_risk_premium

    # 負債コストの計算
    interest_expense_keys = ['支払利息']
    interest_expense = get_value(pl_data['営業外費用'], interest_expense_keys)['当年']
    debt_keys = ['固定負債合計']
    debt = get_value(bs_data['負債の部'], debt_keys)['当年']
    cost_of_debt = interest_expense / debt if debt != 0 else 0

    # 実効税率の計算
    tax_keys = ['法人税等合計']
    pre_tax_profit_keys = ['税金等調整前当期純利益']
    tax_amount = get_value(pl_data['法人税等合計'], tax_keys)['当年']
    pre_tax_profit = get_value(pl_data['税金等調整前当期純利益'], pre_tax_profit_keys)['当年']
    tax_rate = tax_amount / pre_tax_profit if pre_tax_profit != 0 else 0

    # 株主資本と負債の構成比率の計算
    equity_keys = ['純資産合計']
    equity = get_value(bs_data['純資産の部'], equity_keys)['当年']
    total_capital = equity + debt
    equity_ratio = equity / total_capital if total_capital != 0 else 0
    debt_ratio = debt / total_capital if total_capital != 0 else 0

    # WACCの計算
    wacc = cost_of_equity * equity_ratio + cost_of_debt * (1 - tax_rate) * debt_ratio

    return wacc

# JSONファイルからデータを読み込む
with open('bs.json', 'r', encoding='utf-8') as f:
    bs_data = json.load(f)

with open('pl.json', 'r', encoding='utf-8') as f:
    pl_data = json.load(f)

# WACCの計算
wacc = calculate_wacc(bs_data, pl_data)

print(f"WACC: {wacc:.2%}")


WACC: 7.61%


In [10]:

def calculate_ebitda_margin(pl_data, cf_data):
    # EBITDAの計算
    operating_profit_keys = ['営業利益']
    operating_profit = get_value(pl_data['営業利益'], operating_profit_keys)['当年']

    depreciation_keys = ['減価償却費']
    depreciation = get_value(cf_data['営業活動によるキャッシュ・フロー'], depreciation_keys)['当年']
    ebitda = operating_profit + depreciation

    # 売上高の取得
    revenue_keys = ['売上高']
    revenue = get_value(pl_data['売上高'], revenue_keys)['当年']

    # EBITDAマージンの計算
    ebitda_margin = ebitda / revenue if revenue != 0 else 0

    return ebitda_margin

# JSONファイルからデータを読み込む
with open('pl.json', 'r', encoding='utf-8') as f:
    pl_data = json.load(f)
with open('cf.json', 'r', encoding='utf-8') as f:
    cf_data = json.load(f)

# EBITDAマージンの計算
ebitda_margin = calculate_ebitda_margin(pl_data, cf_data)

print(f"EBITDA Margin: {ebitda_margin:.2%}")


EBITDA Margin: 13.69%


In [11]:

def calculate_roic_before_tax(bs_data, pl_data):
    # 営業利益の取得
    operating_profit_keys = ['営業利益']
    operating_profit = get_value(pl_data['営業利益'], operating_profit_keys)['当年']

    # 投下資本の計算
    total_assets_keys = ['資産合計']
    current_liabilities_keys = ['流動負債合計']
    total_assets = get_value(bs_data['資産の部'], total_assets_keys)['当年']
    current_liabilities = get_value(bs_data['負債の部'], current_liabilities_keys)['当年']
    invested_capital = total_assets - current_liabilities

    # 税引き前ROICの計算
    roic_before_tax = operating_profit / invested_capital if invested_capital != 0 else 0

    return roic_before_tax

# JSONファイルからデータを読み込む
with open('bs.json', 'r', encoding='utf-8') as f:
    bs_data = json.load(f)

with open('pl.json', 'r', encoding='utf-8') as f:
    pl_data = json.load(f)

# 税引き前ROICの計算
roic_before_tax = calculate_roic_before_tax(bs_data, pl_data)

print(f"ROIC (Before Tax): {roic_before_tax:.2%}")


ROIC (Before Tax): 42.12%


In [12]:

def calculate_operating_profit_margin(pl_data):
    # 営業利益の取得
    operating_profit_keys = ['営業利益']
    operating_profit = get_value(pl_data['営業利益'], operating_profit_keys)['当年']

    # 売上高の取得
    revenue_keys = ['売上高']
    revenue = get_value(pl_data['売上高'], revenue_keys)['当年']

    # 営業利益率の計算
    operating_profit_margin = operating_profit / revenue if revenue != 0 else 0

    return operating_profit_margin

# JSONファイルからデータを読み込む
with open('pl.json', 'r', encoding='utf-8') as f:
    pl_data = json.load(f)

# 営業利益率の計算
operating_profit_margin = calculate_operating_profit_margin(pl_data)

print(f"Operating Profit Margin: {operating_profit_margin:.2%}")


Operating Profit Margin: 12.59%


In [13]:
import json

def calculate_invested_capital_turnover(bs_data, pl_data):
    # 売上高の取得
    revenue = pl_data['売上高']['売上高']['当年']

    # 投下資本の計算
    total_assets = bs_data['資産の部']['資産合計']['当年']
    current_liabilities = bs_data['負債の部']['流動負債合計']['当年']
    invested_capital = total_assets - current_liabilities

    # 投下資本回転率の計算
    invested_capital_turnover = revenue / invested_capital

    return invested_capital_turnover

# JSONファイルからデータを読み込む
with open('bs.json', 'r', encoding='utf-8') as f:
    bs_data = json.load(f)

with open('pl.json', 'r', encoding='utf-8') as f:
    pl_data = json.load(f)

# 投下資本回転率の計算
invested_capital_turnover = calculate_invested_capital_turnover(bs_data, pl_data)

print(f"Invested Capital Turnover: {invested_capital_turnover:.2f}")

Invested Capital Turnover: 3.35


In [14]:
import json


def calculate_invested_capital_turnover(bs_data, pl_data):
    def get_value(data, key):
        """
        Helper function to safely retrieve values from dictionary based on key.
        Handles cases where keys might not directly map or where nested data might not exist.
        """
        try:
            return data[key]['当年']
        except KeyError:
            return 0  # Assuming the key does not exist and setting default to 0

    # 売上高の取得
    revenue = get_value(pl_data['売上高'], '売上高')

    # 投下資本の計算
    total_assets = get_value(bs_data['資産の部'], '資産合計')
    current_liabilities = get_value(bs_data['負債の部'], '流動負債合計')
    invested_capital = total_assets - current_liabilities

    # Ensure we are not dividing by zero
    if invested_capital == 0:
        return 0

    # 投下資本回転率の計算
    invested_capital_turnover = revenue / invested_capital

    return invested_capital_turnover

# JSONファイルからデータを読み込む
with open('bs.json', 'r', encoding='utf-8') as f:
    bs_data = json.load(f)

with open('pl.json', 'r', encoding='utf-8') as f:
    pl_data = json.load(f)

# 投下資本回転率の計算
invested_capital_turnover = calculate_invested_capital_turnover(bs_data, pl_data)

print(f"Invested Capital Turnover: {invested_capital_turnover:.2f}")


Invested Capital Turnover: 3.35


In [15]:
import json


def calculate_liquidity_on_hand(bs_data, pl_data):
    def get_value(data, keys):
        """
        Helper function to safely retrieve values from dictionary based on keys.
        Handles cases where keys might not directly map or where nested data might not exist.
        """
        for key in keys:
            if key in data:
                return data[key]['当年']
        return 0  # If none of the keys are present, return 0

    # 売上高
    revenue = get_value(pl_data['売上高'], ['売上高'])

    # 現金及び預金の取得
    cash_keys = ['現金及び預金', '電子記録債権']
    cash_and_deposits = sum(get_value(bs_data['資産の部'], [key]) for key in cash_keys)

    # 短期借入金の加算（もし存在すれば）
    short_term_loans = get_value(bs_data['負債の部'], ['短期借入金'])
    cash_and_deposits += short_term_loans

    # 計算された手元流動性を売上高で割る
    liquidity_ratio = cash_and_deposits / revenue if revenue != 0 else 0  # Avoid division by zero

    return liquidity_ratio

# JSONファイルからデータを読み込む
with open('bs.json', 'r', encoding='utf-8') as f:
    bs_data = json.load(f)

with open('pl.json', 'r', encoding='utf-8') as f:
    pl_data = json.load(f)

# 手元流動性の計算
liquidity_on_hand = calculate_liquidity_on_hand(bs_data, pl_data)

print(f"Liquidity on Hand: {liquidity_on_hand:.2f}")


Liquidity on Hand: 0.12


In [16]:
import json



def calculate_current_ratio(bs_data):
    def get_value(data, keys):
        """
        Helper function to safely retrieve values from dictionary based on keys.
        Handles cases where keys might not directly map or where nested data might not exist.
        """
        for key in keys:
            if key in data:
                return data[key]['当年']
        return 0  # If none of the keys are present, return 0

    # 流動資産の取得
    current_assets_keys = ['流動資産合計', '流動資産']  # Some possible variations
    current_assets = get_value(bs_data['資産の部'], current_assets_keys)

    # 流動負債の取得
    current_liabilities_keys = ['流動負債合計', '流動負債']  # Some possible variations
    current_liabilities = get_value(bs_data['負債の部'], current_liabilities_keys)

    # 流動性比率の計算
    current_ratio = current_assets / current_liabilities if current_liabilities != 0 else float('inf')  # Avoid division by zero

    return current_ratio

# JSONファイルからデータを読み込む
with open('bs.json', 'r', encoding='utf-8') as f:
    bs_data = json.load(f)

# 流動性比率の計算
current_ratio = calculate_current_ratio(bs_data)

print(f"Current Ratio: {current_ratio:.2f}")


Current Ratio: 2.14


In [17]:
import json



def calculate_quick_ratio(bs_data):
    def get_value(data, keys):
        """
        Helper function to safely retrieve values from dictionary based on keys.
        Handles cases where keys might not directly map or where nested data might not exist.
        """
        total = 0
        for key in keys:
            if key in data and data[key]['当年'] is not None:  # Ensure the value is not None
                total += data[key]['当年']
        return total

    # Define keys for each type of account
    cash_and_deposits_keys = ['現金及び預金', '現金', '預金']
    notes_and_accounts_receivable_keys = ['受取手形及び売掛金', '受取手形', '売掛金', '電子記録債権']
    current_liabilities_keys = ['流動負債合計', '流動負債']

    # Retrieve values using keys
    cash_and_deposits = get_value(bs_data['資産の部'], cash_and_deposits_keys)
    notes_and_accounts_receivable = get_value(bs_data['資産の部'], notes_and_accounts_receivable_keys)
    current_liabilities = get_value(bs_data['負債の部'], current_liabilities_keys)

    # Assuming no cash equivalents or marketable securities are available
    cash_equivalents_and_securities = 0

    # Calculate the quick ratio
    if current_liabilities == 0:
        return float('inf')  # Handle division by zero by returning infinity
    quick_ratio = (cash_and_deposits + notes_and_accounts_receivable + cash_equivalents_and_securities) / current_liabilities * 100

    return quick_ratio

# Load data from JSON files
with open('bs.json', 'r', encoding='utf-8') as f:
    bs_data = json.load(f)

# Calculate the quick ratio
quick_ratio = calculate_quick_ratio(bs_data)
print(f"Quick Ratio: {quick_ratio:.2f}%")


Quick Ratio: 133.22%


In [18]:
import json

def calculate_tangible_fixed_assets_turnover(bs_data, pl_data):
    # Retrieve revenue
    revenue = pl_data['売上高']['売上高']['当年']

    # Define keys for tangible fixed assets categories
    tangible_fixed_assets_keys = [
        '建物（純額）', '構築物（純額）', '機械及び装置（純額）', '車両運搬具（純額）', '工具、器具及び備品（純額）',
        'リース資産（純額）', '建設仮勘定', '土地'
    ]

    # Retrieve total tangible fixed assets
    tangible_fixed_assets = 0
    for key in tangible_fixed_assets_keys:
        if '資産の部' in bs_data and key in bs_data['資産の部']:
            tangible_fixed_assets += bs_data['資産の部'][key]['当年']

    # Avoid division by zero
    if tangible_fixed_assets == 0:
        return float('inf')

    # Calculate tangible fixed assets turnover ratio
    tangible_fixed_assets_turnover = revenue / tangible_fixed_assets
    return tangible_fixed_assets_turnover

# Load data from JSON files
with open('bs.json', 'r', encoding='utf-8') as f:
    bs_data = json.load(f)

with open('pl.json', 'r', encoding='utf-8') as f:
    pl_data = json.load(f)

# Calculate and print the tangible fixed assets turnover
tangible_fixed_assets_turnover = calculate_tangible_fixed_assets_turnover(bs_data, pl_data)
print(f"Tangible Fixed Assets Turnover: {tangible_fixed_assets_turnover:.2f}")

Tangible Fixed Assets Turnover: 11.39


In [19]:
import json

def calculate_days_sales_outstanding(bs_data, pl_data):
    # 売上高の取得
    revenue = pl_data['売上高']['売上高']['当年']

    # 売上債権の取得
    accounts_receivable = 0
    if '資産の部' in bs_data:
        if '受取手形及び売掛金' in bs_data['資産の部']:
            accounts_receivable += bs_data['資産の部']['受取手形及び売掛金']['当年']
        if '電子記録債権' in bs_data['資産の部']:
            accounts_receivable += bs_data['資産の部']['電子記録債権']['当年']
        if '受取手形、売掛金及び契約資産' in bs_data['資産の部']:
            accounts_receivable += bs_data['資産の部']['受取手形、売掛金及び契約資産']['当年']

    # accounts_receivableが0のときの処理
    if accounts_receivable == 0:
        return 0

    # 売上債権回転率の計算
    accounts_receivable_turnover = revenue / accounts_receivable

    # 売上債権回転日数の計算
    days_sales_outstanding = 365 / accounts_receivable_turnover
    return days_sales_outstanding

# JSONファイルからデータを読み込む
with open('bs.json', 'r', encoding='utf-8') as f:
    bs_data = json.load(f)

with open('pl.json', 'r', encoding='utf-8') as f:
    pl_data = json.load(f)

# 売上債権回転日数の計算
days_sales_outstanding = calculate_days_sales_outstanding(bs_data, pl_data)
print(f"Days Sales Outstanding: {days_sales_outstanding:.2f}")

Days Sales Outstanding: 38.47


In [20]:
import json

def calculate_days_inventory_outstanding_cost(bs_data, pl_data):
    # 売上原価の取得
    cost_of_goods_sold_current = pl_data['売上原価']['売上原価']['当年']
    cost_of_goods_sold_prev = pl_data['売上原価']['売上原価']['前年']

    # 棚卸資産の取得
    inventories_current = 0
    inventories_prev = 0

    # 棚卸資産のカテゴリを指定する
    inventory_items = ['仕掛品', '原材料及び貯蔵品', '商品']  # ここに必要な棚卸資産のカテゴリを追加

    for item in inventory_items:
        if item in bs_data['資産の部']:
            inventories_current += bs_data['資産の部'][item]['当年']
            inventories_prev += bs_data['資産の部'][item]['前年']

    # 期中平均棚卸資産の計算
    average_inventories = (inventories_current + inventories_prev) / 2 if (inventories_current + inventories_prev) > 0 else 1

    # 棚卸資産回転日数（原価）の計算
    inventory_turnover = (cost_of_goods_sold_current + cost_of_goods_sold_prev) / 2 / average_inventories
    days_inventory_outstanding_cost = 365 / inventory_turnover

    return days_inventory_outstanding_cost

# JSONファイルからデータを読み込む
with open('bs.json', 'r', encoding='utf-8') as f:
    bs_data = json.load(f)

with open('pl.json', 'r', encoding='utf-8') as f:
    pl_data = json.load(f)

# 棚卸資産回転日数（原価）の計算
days_inventory_outstanding_cost = calculate_days_inventory_outstanding_cost(bs_data, pl_data)
print(f"Days Inventory Outstanding (Cost): {days_inventory_outstanding_cost:.2f}")


Days Inventory Outstanding (Cost): 45.21


In [21]:
"""
import json

def calculate_days_payable_outstanding(bs_data, pl_data):
    # 売上原価の取得
    cost_of_goods_sold = pl_data['売上原価']['売上原価']['当年']

    # 買入債務の取得
    accounts_payable = bs_data['負債の部']['買掛金']['当年']

    # 買入債務回転率の計算
    accounts_payable_turnover = cost_of_goods_sold / accounts_payable

    # 買入債務回転日数の計算
    days_payable_outstanding = 365 / accounts_payable_turnover

    return days_payable_outstanding

# JSONファイルからデータを読み込む
with open('bs.json', 'r', encoding='utf-8') as f:
    bs_data = json.load(f)

with open('pl.json', 'r', encoding='utf-8') as f:
    pl_data = json.load(f)

# 買入債務回転日数の計算
days_payable_outstanding = calculate_days_payable_outstanding(bs_data, pl_data)

print(f"Days Payable Outstanding: {days_payable_outstanding:.2f}")
"""

'\nimport json\n\ndef calculate_days_payable_outstanding(bs_data, pl_data):\n    # 売上原価の取得\n    cost_of_goods_sold = pl_data[\'売上原価\'][\'売上原価\'][\'当年\']\n\n    # 買入債務の取得\n    accounts_payable = bs_data[\'負債の部\'][\'買掛金\'][\'当年\']\n\n    # 買入債務回転率の計算\n    accounts_payable_turnover = cost_of_goods_sold / accounts_payable\n\n    # 買入債務回転日数の計算\n    days_payable_outstanding = 365 / accounts_payable_turnover\n\n    return days_payable_outstanding\n\n# JSONファイルからデータを読み込む\nwith open(\'bs.json\', \'r\', encoding=\'utf-8\') as f:\n    bs_data = json.load(f)\n\nwith open(\'pl.json\', \'r\', encoding=\'utf-8\') as f:\n    pl_data = json.load(f)\n\n# 買入債務回転日数の計算\ndays_payable_outstanding = calculate_days_payable_outstanding(bs_data, pl_data)\n\nprint(f"Days Payable Outstanding: {days_payable_outstanding:.2f}")\n'

In [22]:
"""
claude3の回答。38.02の結果がほしいが46.49となるため、目標とする結果よりこのプログラムのほうが優れている点を質問した。
---------------
私が提供したプログラムの方が、より正確な買入債務回転日数を計算できる理由は主に以下の点にあります。

買入債務の計算方法 私のプログラムでは、買入債務の計算に「買掛金」だけでなく、「未払金」と「未払費用」も含めています。単に買掛金のみを使うと、買入債務を過小評価してしまう可能性があります。
期中平均の使用 私のプログラムでは、当年度と前年度の買入債務の平均値を使用しています。これにより、期中の変動を考慮した、より適切な買入債務の値を使うことができます。期末残高のみを使うと、一時的な変動の影響を受けやすくなります。
柔軟性 私のプログラムでは、関数化することで、売上原価と買入債務の計算方法を柔軟に変更できるようになっています。企業によって会計方針が異なる場合にも、適切な計算方法を実装できます。
エラー処理 私のプログラムでは、キーが存在しない場合や、買入債務が0の場合の適切な処理を行っています。これにより、エラーが発生せずに計算を続行できます。
これらの点から、私のプログラムは、より正確かつ柔軟な計算ができると考えられます。ただし、実際のデータや会計方針によっては、さらなる調整が必要な場合もあります。

提示されたプログラムは、単純化されているため、一定の条件下では適切な結果を出すかもしれませんが、一般的には私のプログラムの方が優れていると言えます。特に、企業の実情に合わせた適切な計算方法を実装できる点が大きな利点となります。


"""
import json

def calculate_days_payable_outstanding(bs_data, pl_data):
    # 売上原価の取得または計算
    cost_of_goods_sold = calculate_cost_of_goods_sold(pl_data)

    # 買入債務の取得
    accounts_payable_current = calculate_accounts_payable(bs_data, '当年')
    accounts_payable_prev = calculate_accounts_payable(bs_data, '前年')
    accounts_payable_avg = (accounts_payable_current + accounts_payable_prev) / 2

    # accounts_payableが0のときの処理
    if accounts_payable_avg == 0:
        return 0

    # 買入債務回転率の計算
    accounts_payable_turnover = cost_of_goods_sold / accounts_payable_avg

    # 買入債務回転日数の計算
    days_payable_outstanding = 365 / accounts_payable_turnover
    return days_payable_outstanding

def calculate_cost_of_goods_sold(pl_data):
    # 売上原価の取得または計算ロジックを追加
    cost_of_goods_sold = pl_data['売上原価']['売上原価']['当年']
    return cost_of_goods_sold

def calculate_accounts_payable(bs_data, year):
    # 買入債務の取得または計算ロジックを追加
    accounts_payable = bs_data['負債の部']['買掛金'][year]
    if '未払金' in bs_data['負債の部']:
        accounts_payable += bs_data['負債の部']['未払金'][year]
    if '未払費用' in bs_data['負債の部']:
        accounts_payable += bs_data['負債の部']['未払費用'][year]
    return accounts_payable

# JSONファイルからデータを読み込む
with open('bs.json', 'r', encoding='utf-8') as f:
    bs_data = json.load(f)

with open('pl.json', 'r', encoding='utf-8') as f:
    pl_data = json.load(f)

# 買入債務回転日数の計算
days_payable_outstanding = calculate_days_payable_outstanding(bs_data, pl_data)
print(f"Days Payable Outstanding: {days_payable_outstanding:.2f}")

Days Payable Outstanding: 46.49


In [23]:
import json



def calculate_metrics(bs_data, pl_data):
    def find_value(data, keys):
        """ This function attempts to find and return the first available value from the data using a list of keys. """
        for key in keys:
            if key in data:
                value = data[key]
                if isinstance(value, dict) and '当年' in value:
                    return value['当年']
                elif isinstance(value, (int, float)):  # Direct match with a numeric type
                    return value
        return 0
    # Possible keys from the provided accounts
    receivable_keys = [
        '受取手形及び売掛金', '受取手形及び売掛金（純額）', '売掛金', '売掛金（純額）', '受取手形',
        '受取手形（純額）', '関係会社売掛金', '割賦売掛金'
    ]
    inventory_keys = [
        '棚卸資産', '商品', '製品', '半製品', '原材料及び貯蔵品', '原材料', '仕掛品', '完成業務未収入金'
    ]
    payable_keys = [
        '買掛金', '支払手形及び買掛金'
    ]

    # Extracting data
    revenue = pl_data.get('売上高', {}).get('売上高', {}).get('当年', 0)
    cost_of_goods_sold = pl_data.get('売上原価', {}).get('売上原価', {}).get('当年', 0)

    receivables = sum(find_value(bs_data['資産の部'], [key]) for key in receivable_keys)
    inventories = sum(find_value(bs_data['資産の部'], [key]) for key in inventory_keys)
    payables = sum(find_value(bs_data['負債の部'], [key]) for key in payable_keys)

    # Calculations
    days_sales_outstanding = (365 * receivables / revenue) if revenue else float('inf')
    days_inventory_outstanding = (365 * inventories / cost_of_goods_sold) if cost_of_goods_sold else float('inf')
    days_payable_outstanding = (365 * payables / cost_of_goods_sold) if cost_of_goods_sold else float('inf')

    cash_conversion_cycle = days_sales_outstanding + days_inventory_outstanding - days_payable_outstanding

    return days_sales_outstanding, days_inventory_outstanding, days_payable_outstanding, cash_conversion_cycle

# Load data
with open('bs.json', 'r', encoding='utf-8') as f:
    bs_data = json.load(f)

with open('pl.json', 'r', encoding='utf-8') as f:
    pl_data = json.load(f)

# Cash Conversion Cycleの計算
# Calculate metrics
dso, dio, dpo, ccc = calculate_metrics(bs_data, pl_data)
print(f"Days Sales Outstanding: {dso:.2f} days")
print(f"Days Inventory Outstanding: {dio:.2f} days")
print(f"Days Payable Outstanding: {dpo:.2f} days")
print(f"Cash Conversion Cycle: {ccc:.2f} days")


Days Sales Outstanding: 37.36 days
Days Inventory Outstanding: 44.39 days
Days Payable Outstanding: 39.17 days
Cash Conversion Cycle: 42.58 days


In [24]:
import json

def calculate_effective_tax_rate(pl_data):
    # 税金等調整前当期純利益の取得
    profit_before_tax = pl_data['税金等調整前当期純利益']['税金等調整前当期純利益']['当年']

    # 法人税、住民税及び事業税の取得
    corporate_tax = pl_data['法人税、住民税及び事業税']['法人税、住民税及び事業税']['当年']

    # 法人税等調整額の取得
    deferred_tax = pl_data['法人税等調整額']['法人税等調整額']['当年']

    # 税率の計算
    effective_tax_rate = (corporate_tax + deferred_tax) / profit_before_tax

    return effective_tax_rate

# JSONファイルからデータを読み込む
with open('pl.json', 'r', encoding='utf-8') as f:
    pl_data = json.load(f)

# 税率の計算
effective_tax_rate = calculate_effective_tax_rate(pl_data)

print(f"Effective Tax Rate: {effective_tax_rate:.2%}")

Effective Tax Rate: 31.96%


In [25]:
import json



def calculate_noplat(pl_data):
    def get_value(data, key):
        # Safely retrieves value if key exists and is a dictionary with '当年' key, else returns 0
        return data.get(key, {}).get('当年', 0)
    # Extract required data using safe get_value function
    operating_income = get_value(pl_data['営業利益'], '営業利益')
    interest_expenses = get_value(pl_data['営業外費用'], '支払利息')
    pretax_income = get_value(pl_data['税金等調整前当期純利益'], '税金等調整前当期純利益')
    total_tax = get_value(pl_data['法人税等合計'], '法人税等合計')

    # Calculate the effective tax rate, handling division by zero
    tax_rate = total_tax / pretax_income if pretax_income != 0 else 0

    # Calculate NOPLAT, adding back tax-affected interest expenses
    noplat = operating_income - (operating_income * tax_rate) + (interest_expenses * tax_rate)

    # Outputting results
    print(f"Effective Tax Rate: {tax_rate:.2f}")
    return noplat

# Loading data from JSON files
with open('pl.json', 'r', encoding='utf-8') as f:
    pl_data = json.load(f)

# Calculating NOPLAT
noplat = calculate_noplat(pl_data)

print(f"NOPLAT: {noplat:.2f}")


Effective Tax Rate: 0.32
NOPLAT: 9404705513.18


In [26]:
import json

def calculate_non_cash_expenses(pl_data, cf_data):
    # 減価償却費の取得
    depreciation = cf_data['営業活動によるキャッシュ・フロー']['減価償却費']['当年']

    # 貸倒引当金の増減額の取得
    provision_for_doubtful_accounts = cf_data['営業活動によるキャッシュ・フロー']['貸倒引当金の増減額（△は減少）']['当年']

    # 賞与引当金の増減額の取得
    provision_for_bonuses = cf_data['営業活動によるキャッシュ・フロー']['賞与引当金の増減額（△は減少）']['当年']

    # 退職給付に係る負債の増減額の取得
    net_defined_benefit_liability = cf_data['営業活動によるキャッシュ・フロー']['退職給付に係る負債の増減額（△は減少）']['当年']

    # 非現金支出の計算
    non_cash_expenses = depreciation + provision_for_doubtful_accounts + provision_for_bonuses + net_defined_benefit_liability

    return non_cash_expenses

# JSONファイルからデータを読み込む
with open('pl.json', 'r', encoding='utf-8') as f:
    pl_data = json.load(f)

with open('cf.json', 'r', encoding='utf-8') as f:
    cf_data = json.load(f)

# 非現金支出の計算
non_cash_expenses = calculate_non_cash_expenses(pl_data, cf_data)

print(f"Non-Cash Expenses: {non_cash_expenses:,}")

Non-Cash Expenses: 1,517,131,000


In [27]:
def calculate_accounts_receivable(bs_data):
    # 受取手形及び売掛金の取得
    notes_and_accounts_receivable_trade = 0
    if '資産の部' in bs_data and '受取手形及び売掛金' in bs_data['資産の部']:
        notes_and_accounts_receivable_trade = bs_data['資産の部']['受取手形及び売掛金']['当年']

    # 電子記録債権の取得
    electronically_recorded_monetary_claims_operating = 0
    if '資産の部' in bs_data and '電子記録債権' in bs_data['資産の部']:
        electronically_recorded_monetary_claims_operating = bs_data['資産の部']['電子記録債権']['当年']

    # 受取手形、売掛金及び契約資産の取得
    accounts_receivable_trade = 0
    if '資産の部' in bs_data and '受取手形、売掛金及び契約資産' in bs_data['資産の部']:
        accounts_receivable_trade = bs_data['資産の部']['受取手形、売掛金及び契約資産']['当年']

    # 売上債権の計算
    accounts_receivable = notes_and_accounts_receivable_trade + electronically_recorded_monetary_claims_operating + accounts_receivable_trade

    return accounts_receivable

def calculate_days_sales_outstanding(bs_data, pl_data):
    if '売上高' in pl_data and '売上高' in pl_data['売上高']:
        revenue = pl_data['売上高']['売上高']['当年']
    else:
        return 0

    accounts_receivable = calculate_accounts_receivable(bs_data)
    if accounts_receivable == 0:
        return 0

    accounts_receivable_turnover = revenue / accounts_receivable
    days_sales_outstanding = 365 / accounts_receivable_turnover
    return days_sales_outstanding

def calculate_days_inventory_outstanding_cost(bs_data, pl_data):
    if '売上原価' in pl_data and '売上原価' in pl_data['売上原価']:
        cost_of_goods_sold = pl_data['売上原価']['売上原価']['当年']
    else:
        return 0

    inventories_current = 0
    inventories_prev = 0

    if '資産の部' in bs_data:
        if '仕掛品' in bs_data['資産の部']:
            inventories_current += bs_data['資産の部']['仕掛品']['当年']
            inventories_prev += bs_data['資産の部']['仕掛品']['前年']

        if '原材料及び貯蔵品' in bs_data['資産の部']:
            inventories_current += bs_data['資産の部']['原材料及び貯蔵品']['当年']
            inventories_prev += bs_data['資産の部']['原材料及び貯蔵品']['前年']

        if '商品' in bs_data['資産の部']:
            inventories_current += bs_data['資産の部']['商品']['当年']
            inventories_prev += bs_data['資産の部']['商品']['前年']

    average_inventories = (inventories_current + inventories_prev) / 2
    if average_inventories == 0:
        return 0

    inventory_turnover_cost = cost_of_goods_sold / average_inventories
    days_inventory_outstanding_cost = 365 / inventory_turnover_cost
    return days_inventory_outstanding_cost

def calculate_days_payable_outstanding(bs_data, pl_data):
    if '売上原価' in pl_data and '売上原価' in pl_data['売上原価']:
        cost_of_goods_sold = pl_data['売上原価']['売上原価']['当年']
    else:
        return 0

    accounts_payable = 0
    if '負債の部' in bs_data and '買掛金' in bs_data['負債の部']:
        accounts_payable = bs_data['負債の部']['買掛金']['当年']

    if accounts_payable == 0:
        return 0

    accounts_payable_turnover = cost_of_goods_sold / accounts_payable
    days_payable_outstanding = 365 / accounts_payable_turnover
    return days_payable_outstanding

def calculate_cash_conversion_cycle(bs_data, pl_data):
    days_sales_outstanding = calculate_days_sales_outstanding(bs_data, pl_data)
    days_inventory_outstanding_cost = calculate_days_inventory_outstanding_cost(bs_data, pl_data)
    days_payable_outstanding = calculate_days_payable_outstanding(bs_data, pl_data)
    cash_conversion_cycle = days_sales_outstanding + days_inventory_outstanding_cost - days_payable_outstanding
    return cash_conversion_cycle

# JSONファイルからデータを読み込む
with open('bs.json', 'r', encoding='utf-8') as f:
    bs_data = json.load(f)

with open('pl.json', 'r', encoding='utf-8') as f:
    pl_data = json.load(f)

# 売上債権の計算
accounts_receivable = calculate_accounts_receivable(bs_data)
print(f"売上債権: {accounts_receivable:,.0f}円")

# Cash Conversion Cycleの計算
cash_conversion_cycle = calculate_cash_conversion_cycle(bs_data, pl_data)
print(f"Cash Conversion Cycle: {cash_conversion_cycle:.2f} days")

売上債権: 11,546,872,000円
Cash Conversion Cycle: 39.82 days


In [28]:
import json

def calculate_inventory(bs_data):
    # 商品の取得
    merchandise = 0
    if '資産の部' in bs_data and '商品' in bs_data['資産の部']:
        merchandise = bs_data['資産の部']['商品']['当年']

    # 未着商品の取得
    merchandise_in_transit = 0
    if '資産の部' in bs_data and '未着商品' in bs_data['資産の部']:
        merchandise_in_transit = bs_data['資産の部']['未着商品']['当年']

    # 貯蔵品の取得
    supplies = 0
    if '資産の部' in bs_data and '貯蔵品' in bs_data['資産の部']:
        supplies = bs_data['資産の部']['貯蔵品']['当年']

    # 在庫の計算
    inventory = merchandise + merchandise_in_transit + supplies
    return inventory

# JSONファイルからデータを読み込む
with open('bs.json', 'r', encoding='utf-8') as f:
    bs_data = json.load(f)

# 在庫の計算
inventory = calculate_inventory(bs_data)
print(f"Inventory: {inventory:,}")

Inventory: 9,987,055,000.0


In [29]:
import json

def calculate_accounts_payable(bs_data):
    # 買掛金の取得
    accounts_payable_trade = 0
    if '負債の部' in bs_data and '買掛金' in bs_data['負債の部']:
        accounts_payable_trade = bs_data['負債の部']['買掛金']['当年']

    # 未払金の取得
    accounts_payable_other = 0
    if '負債の部' in bs_data and '未払金' in bs_data['負債の部']:
        accounts_payable_other = bs_data['負債の部']['未払金']['当年']

    # 支払債務の計算
    accounts_payable = accounts_payable_trade + accounts_payable_other
    return accounts_payable

# JSONファイルからデータを読み込む
with open('bs.json', 'r', encoding='utf-8') as f:
    bs_data = json.load(f)

# 支払債務の計算
accounts_payable = calculate_accounts_payable(bs_data)
print(f"Accounts Payable: {accounts_payable:,}")

Accounts Payable: 10,864,205,000.0


In [30]:
import json

def calculate_working_capital(bs_data):
    # 流動資産を計算
    cash_and_deposits = bs_data['資産の部']['現金及び預金']['当年'] if '現金及び預金' in bs_data['資産の部'] else 0
    notes_and_accounts_receivable = (bs_data['資産の部']['受取手形及び売掛金']['当年'] if '受取手形及び売掛金' in bs_data['資産の部'] else 0) + \
                                     (bs_data['資産の部']['電子記録債権']['当年'] if '電子記録債権' in bs_data['資産の部'] else 0) + \
                                     (bs_data['資産の部']['受取手形、売掛金及び契約資産']['当年'] if '受取手形、売掛金及び契約資産' in bs_data['資産の部'] else 0)
    merchandise = bs_data['資産の部']['商品']['当年'] if '商品' in bs_data['資産の部'] else 0
    merchandise_in_transit = bs_data['資産の部']['未着商品']['当年'] if '未着商品' in bs_data['資産の部'] else 0
    supplies = bs_data['資産の部']['貯蔵品']['当年'] if '貯蔵品' in bs_data['資産の部'] else 0
    other_current_assets = bs_data['資産の部']['その他']['当年'] if 'その他' in bs_data['資産の部'] else 0
    current_assets = cash_and_deposits + notes_and_accounts_receivable + merchandise + merchandise_in_transit + supplies

    # 流動負債を計算
    accounts_payable = bs_data['負債の部']['買掛金']['当年'] if '買掛金' in bs_data['負債の部'] else 0
    if '未払金' in bs_data['負債の部']:
        accounts_payable += bs_data['負債の部']['未払金']['当年']
    short_term_debt = (bs_data['負債の部']['1年内返済予定の長期借入金']['当年'] if '1年内返済予定の長期借入金' in bs_data['負債の部'] else 0) + \
                       (bs_data['負債の部']['リース債務']['当年'] if 'リース債務' in bs_data['負債の部'] else 0)
    accrued_expenses = (bs_data['負債の部']['未払法人税等']['当年'] if '未払法人税等' in bs_data['負債の部'] else 0) + \
                        (bs_data['負債の部']['賞与引当金']['当年'] if '賞与引当金' in bs_data['負債の部'] else 0) + \
                        (bs_data['負債の部']['役員賞与引当金']['当年'] if '役員賞与引当金' in bs_data['負債の部'] else 0) + \
                        (bs_data['負債の部']['事故関連損失引当金']['当年'] if '事故関連損失引当金' in bs_data['負債の部'] else 0)
    current_liabilities = accounts_payable + short_term_debt + accrued_expenses

    # working capitalを計算
    working_capital = current_assets - current_liabilities
    return working_capital

# JSONファイルからデータを読み込む
with open('bs.json', 'r', encoding='utf-8') as f:
    bs_data = json.load(f)

# Working Capitalの計算
working_capital = calculate_working_capital(bs_data)
print(f"Working Capital: {working_capital:,}")

Working Capital: 18,061,560,000.0


In [31]:
import json

def calculate_capital_expenditure(cf_data):
    # 有形固定資産の取得による支出の取得
    purchase_of_property_plant_and_equipment = cf_data['投資活動によるキャッシュ・フロー']['有形固定資産の取得による支出']['当年']

    # 無形固定資産の取得による支出の取得
    purchase_of_intangible_assets = cf_data['投資活動によるキャッシュ・フロー']['無形固定資産の取得による支出']['当年']

    # 設備投資の計算
    capital_expenditure = purchase_of_property_plant_and_equipment + purchase_of_intangible_assets

    return capital_expenditure

# JSONファイルからデータを読み込む
with open('cf.json', 'r', encoding='utf-8') as f:
    cf_data = json.load(f)

# 設備投資の計算
capital_expenditure = calculate_capital_expenditure(cf_data)

print(f"Capital Expenditure: {capital_expenditure:,}")

Capital Expenditure: -1,053,660,000


In [32]:
import json

def get_value(data, keys):
    """Safely retrieve data using a list of keys; if a key is missing, output a debug message and return 0."""
    for key in keys:
        if key in data:
            data = data[key]
        else:
            print(f"Key missing: {key}")  # Debug print for missing key
            return 0  # Return 0 if any key is missing
    return data if isinstance(data, float) else data.get('当年', 0)  # Check type before using get

def calculate_days_sales_outstanding(bs_data, pl_data):
    """Calculate Days Sales Outstanding (DSO) using expanded account items."""
    net_sales = get_value(pl_data, ['売上高', '売上高'])  # Placeholder keys, adjust according to actual data structure
    if net_sales == 0:
        return float('inf')  # Avoid division by zero
    # Expanded accounts receivable items
    accounts_receivable_keys = [
        '受取手形及び売掛金', '電子記録債権', '受取手形、売掛金及び契約資産', '売掛金'
    ]
    average_accounts_receivable = sum(get_value(bs_data.get('資産の部', {}), [key]) for key in accounts_receivable_keys)
    days_sales_outstanding = 365 * average_accounts_receivable / net_sales
    return days_sales_outstanding

def calculate_days_inventory_outstanding_cost(bs_data, pl_data):
    """Revised calculation for Days Inventory Outstanding to include all relevant inventory accounts."""
    cost_of_goods_sold = get_value(pl_data, ['売上原価', '売上原価'])
    inventory_keys = ['仕掛品', '原材料及び貯蔵品', '商品']
    inventories = sum(get_value(bs_data.get('資産の部', {}), [key]) for key in inventory_keys if key in bs_data.get('資産の部', {}))
    if cost_of_goods_sold == 0 or inventories == 0:
        return float('inf')  # Return infinity if no COGS or inventory to avoid division by zero
    inventory_turnover_cost = cost_of_goods_sold / inventories
    days_inventory_outstanding_cost = 365 / inventory_turnover_cost
    return days_inventory_outstanding_cost

def calculate_days_payable_outstanding(bs_data, pl_data):
    """Revised calculation for Days Payable Outstanding to include all relevant payable accounts."""
    cost_of_goods_sold = get_value(pl_data, ['売上原価', '売上原価'])
    accounts_payable_keys = ['買掛金', '未払金']
    accounts_payable = sum(get_value(bs_data.get('負債の部', {}), [key]) for key in accounts_payable_keys if key in bs_data.get('負債の部', {}))
    if cost_of_goods_sold == 0 or accounts_payable == 0:
        return float('inf')  # Return infinity if no COGS or accounts payable to avoid division by zero
    accounts_payable_turnover = cost_of_goods_sold / accounts_payable
    days_payable_outstanding = 365 / accounts_payable_turnover
    return days_payable_outstanding

def calculate_cash_conversion_cycle(bs_data, pl_data):
    days_sales_outstanding = calculate_days_sales_outstanding(bs_data, pl_data)
    days_inventory_outstanding_cost = calculate_days_inventory_outstanding_cost(bs_data, pl_data)
    days_payable_outstanding = calculate_days_payable_outstanding(bs_data, pl_data)
    cash_conversion_cycle = days_sales_outstanding + days_inventory_outstanding_cost - days_payable_outstanding
    return cash_conversion_cycle

# Load JSON data and calculate CCC
with open('bs.json', 'r', encoding='utf-8') as f:
    bs_data = json.load(f)
with open('pl.json', 'r', encoding='utf-8') as f:
    pl_data = json.load(f)

cash_conversion_cycle = calculate_cash_conversion_cycle(bs_data, pl_data)
print(f"Cash Conversion Cycle: {cash_conversion_cycle:.2f} days")


Key missing: 受取手形、売掛金及び契約資産
Key missing: 売掛金
Cash Conversion Cycle: 31.71 days


In [33]:
import json



def calculate_working_capital(bs_data, year):
    # 受取手形及び売掛金の取得
    notes_and_accounts_receivable_trade = 0
    if '資産の部' in bs_data and '受取手形及び売掛金' in bs_data['資産の部']:
        notes_and_accounts_receivable_trade = bs_data['資産の部']['受取手形及び売掛金'][year]

    # 電子記録債権の取得
    electronically_recorded_monetary_claims_operating = 0
    if '資産の部' in bs_data and '電子記録債権' in bs_data['資産の部']:
        electronically_recorded_monetary_claims_operating = bs_data['資産の部']['電子記録債権'][year]

    # 売上債権の計算
    accounts_receivable = notes_and_accounts_receivable_trade + electronically_recorded_monetary_claims_operating

    # 商品の取得
    merchandise = 0
    if '資産の部' in bs_data and '商品' in bs_data['資産の部']:
        merchandise = bs_data['資産の部']['商品'][year]

    # 未着商品の取得
    merchandise_in_transit = 0
    if '資産の部' in bs_data and '未着商品' in bs_data['資産の部']:
        merchandise_in_transit = bs_data['資産の部']['未着商品'][year]

    # 貯蔵品の取得
    supplies = 0
    if '資産の部' in bs_data and '貯蔵品' in bs_data['資産の部']:
        supplies = bs_data['資産の部']['貯蔵品'][year]

    # 在庫の計算
    inventory = merchandise + merchandise_in_transit + supplies

    # 買掛金の取得
    accounts_payable_trade = 0
    if '負債の部' in bs_data and '買掛金' in bs_data['負債の部']:
        accounts_payable_trade = bs_data['負債の部']['買掛金'][year]

    # 未払金の取得
    accounts_payable_other = 0
    if '負債の部' in bs_data and '未払金' in bs_data['負債の部']:
        accounts_payable_other = bs_data['負債の部']['未払金'][year]

    # 支払債務の計算
    accounts_payable = accounts_payable_trade + accounts_payable_other

    working_capital = accounts_receivable + inventory - accounts_payable
    return working_capital


def calculate_free_cash_flow(pl_data, cf_data, bs_data):
    noplat = calculate_noplat(pl_data)
    non_cash_expenses = calculate_non_cash_expenses(pl_data, cf_data)
    working_capital_change = calculate_working_capital(bs_data, '当年') - calculate_working_capital(bs_data, '前年')
    capital_expenditure = calculate_capital_expenditure(cf_data)

    free_cash_flow = noplat + non_cash_expenses - working_capital_change - capital_expenditure

    return free_cash_flow

# JSONファイルからデータを読み込む
with open('pl.json', 'r', encoding='utf-8') as f:
    pl_data = json.load(f)

with open('cf.json', 'r', encoding='utf-8') as f:
    cf_data = json.load(f)

with open('bs.json', 'r', encoding='utf-8') as f:
    bs_data = json.load(f)

# Free Cash Flowの計算
free_cash_flow = calculate_free_cash_flow(pl_data, cf_data, bs_data)

print(f"Free Cash Flow: {free_cash_flow:,}")

Effective Tax Rate: 0.32
Free Cash Flow: 10,543,831,513.180286


In [34]:
from lxml import etree
import re
import datetime

english_to_japanese = {
    "DocumentTitle": "提出書類",
    "ClauseOfStipulation": "根拠条文",
    "PlaceOfFiling": "提出先",
    "FilingDate": "提出日",
    "FiscalYear": "事業年度",
    "CompanyName": "会社名",
    "CompanyNameInEnglish": "英訳名",
    "RepresentativeTitleAndName": "代表者の役職氏名",
    "RegisteredHeadquarterAddress": "本店の所在の場所",
    "HeadquarterTelephone": "電話番号",
    "ContactPerson": "事務連絡者氏名",
    "NearestContactAddress": "最寄りの連絡場所",
    "NearestContactTelephone": "電話番号",  # If needed for the nearest contact telephone as well
    "NearestContactPerson": "最寄りの連絡者氏名",
    "PlaceForPublicInspection": "縦覧に供する場所",
    "FiscalYearStartDate":"会計年度開始年月日",
    "FiscalYearEndDate":"会計年度終了年月日",
}

def extract_information_from_xbrl(file_path):
    period,start_date,end_date = None, None,None
    # Load and parse the XBRL XML file using lxml
    tree = etree.parse(file_path)
    root = tree.getroot()

    # Namespace map for XPath
    nsmap = root.nsmap

    # Dictionary to store extracted information using XPath
    extracted_info = {
        "DocumentTitle": root.xpath('//jpcrp_cor:DocumentTitleCoverPage', namespaces=nsmap)[0].text if root.xpath('//jpcrp_cor:DocumentTitleCoverPage', namespaces=nsmap) else "Not Found",
        "ClauseOfStipulation": root.xpath('//jpcrp_cor:ClauseOfStipulationCoverPage', namespaces=nsmap)[0].text if root.xpath('//jpcrp_cor:ClauseOfStipulationCoverPage', namespaces=nsmap) else "Not Found",
        "PlaceOfFiling": root.xpath('//jpcrp_cor:PlaceOfFilingCoverPage', namespaces=nsmap)[0].text if root.xpath('//jpcrp_cor:PlaceOfFilingCoverPage', namespaces=nsmap) else "Not Found",
        "FilingDate": root.xpath('//jpcrp_cor:FilingDateCoverPage', namespaces=nsmap)[0].text if root.xpath('//jpcrp_cor:FilingDateCoverPage', namespaces=nsmap) else "Not Found",
        "FiscalYear": root.xpath('//jpcrp_cor:FiscalYearCoverPage', namespaces=nsmap)[0].text if root.xpath('//jpcrp_cor:FiscalYearCoverPage', namespaces=nsmap) else "Not Found",
        "CompanyName": root.xpath('//jpcrp_cor:CompanyNameCoverPage', namespaces=nsmap)[0].text if root.xpath('//jpcrp_cor:CompanyNameCoverPage', namespaces=nsmap) else "Not Found",
        "CompanyNameInEnglish": root.xpath('//jpcrp_cor:CompanyNameInEnglishCoverPage', namespaces=nsmap)[0].text if root.xpath('//jpcrp_cor:CompanyNameInEnglishCoverPage', namespaces=nsmap) else "Not Found",
        "RepresentativeTitleAndName": root.xpath('//jpcrp_cor:TitleAndNameOfRepresentativeCoverPage', namespaces=nsmap)[0].text if root.xpath('//jpcrp_cor:TitleAndNameOfRepresentativeCoverPage', namespaces=nsmap) else "Not Found",
        "RegisteredHeadquarterAddress": root.xpath('//jpcrp_cor:AddressOfRegisteredHeadquarterCoverPage', namespaces=nsmap)[0].text if root.xpath('//jpcrp_cor:AddressOfRegisteredHeadquarterCoverPage', namespaces=nsmap) else "Not Found",
        "HeadquarterTelephone": root.xpath('//jpcrp_cor:TelephoneNumberAddressOfRegisteredHeadquarterCoverPage', namespaces=nsmap)[0].text if root.xpath('//jpcrp_cor:TelephoneNumberAddressOfRegisteredHeadquarterCoverPage', namespaces=nsmap) else "Not Found",
        "ContactPerson": root.xpath('//jpcrp_cor:NameOfContactPersonAddressOfRegisteredHeadquarterCoverPage', namespaces=nsmap)[0].text if root.xpath('//jpcrp_cor:NameOfContactPersonAddressOfRegisteredHeadquarterCoverPage', namespaces=nsmap) else "Not Found",
        "NearestContactAddress": root.xpath('//jpcrp_cor:NearestPlaceOfContactCoverPage', namespaces=nsmap)[0].text if root.xpath('//jpcrp_cor:NearestPlaceOfContactCoverPage', namespaces=nsmap) else "Not Found",
        "NearestContactTelephone": root.xpath('//jpcrp_cor:TelephoneNumberNearestPlaceOfContactCoverPage', namespaces=nsmap)[0].text if root.xpath('//jpcrp_cor:TelephoneNumberNearestPlaceOfContactCoverPage', namespaces=nsmap) else "Not Found",
        "NearestContactPerson": root.xpath('//jpcrp_cor:NameOfContactPersonNearestPlaceOfContactCoverPage', namespaces=nsmap)[0].text if root.xpath('//jpcrp_cor:NameOfContactPersonNearestPlaceOfContactCoverPage', namespaces=nsmap) else "Not Found",
        "PlaceForPublicInspection": " ".join(root.xpath('//jpcrp_cor:PlaceForPublicInspectionCoverPageTextBlock//p/text()', namespaces=nsmap)) ,
        "FiscalYearStartDate": root.xpath('//jpdei_cor:CurrentFiscalYearStartDateDEI', namespaces=nsmap)[0].text if root.xpath('//jpdei_cor:CurrentFiscalYearStartDateDEI', namespaces=nsmap) else "Not Found",
        "FiscalYearEndDate": root.xpath('//jpdei_cor:CurrentFiscalYearEndDateDEI', namespaces=nsmap)[0].text if root.xpath('//jpdei_cor:CurrentFiscalYearEndDateDEI', namespaces=nsmap) else "Not Found",
        }

    return extracted_info


info = extract_information_from_xbrl(xbrlfile)
for key,val in info.items():
  print(key,english_to_japanese[key],val)

# 正規表現パターン
#pattern = r"第(\d+?)期.*?自\s+(\d+)年([０-９1-9]+)月([０-９1-9]+)日\s+至\s+(\d+)年([０-９1-9]+)月([０-９1-9]+)日"
pattern = r"第(\d+)期\s*（自\s*(\d{4})年(\d{1,2})月(\d{1,2})日\s*至\s*(\d{4})年(\d{1,2})月(\d{1,2})日）"

# 検索実行
match = re.search(pattern, info['FiscalYear'])
if match:
    period = match.group(1)
    st_year = match.group(2)
    st_mon = match.group(3)
    st_day = match.group(4)
    end_year = match.group(5)
    end_mon = match.group(6)
    end_day = match.group(7)
    start_date = datetime.datetime(int(st_year),int(st_mon),int(st_day))
    end_date   = datetime.datetime(int(end_year),int(end_mon),int(end_day))
    print(f"会期: {period}, 自の年月日:{start_date}, 至の年月日: {end_date}")
else:
    print("該当するデータが見つかりませんでした。")
print('fiscal start',info['FiscalYearStartDate'])
print('fiscal end',info['FiscalYearEndDate'])


DocumentTitle 提出書類 有価証券報告書
ClauseOfStipulation 根拠条文 金融商品取引法第24条第１項
PlaceOfFiling 提出先 近畿財務局長
FilingDate 提出日 2019-03-26
FiscalYear 事業年度 第19期（自  2018年１月１日  至  2018年12月31日）
CompanyName 会社名 株式会社ＭｏｎｏｔａＲＯ
CompanyNameInEnglish 英訳名 MonotaRO Co.,Ltd.
RepresentativeTitleAndName 代表者の役職氏名 代表執行役社長    鈴木  雅哉
RegisteredHeadquarterAddress 本店の所在の場所 兵庫県尼崎市竹谷町二丁目183番地
HeadquarterTelephone 電話番号 06－4869－7111
ContactPerson 事務連絡者氏名 常務執行役管理部門長    甲田　哲也
NearestContactAddress 最寄りの連絡場所 兵庫県尼崎市竹谷町二丁目183番地
NearestContactTelephone 電話番号 06－4869－7111
NearestContactPerson 最寄りの連絡者氏名 常務執行役管理部門長    甲田　哲也
PlaceForPublicInspection 縦覧に供する場所 
FiscalYearStartDate 会計年度開始年月日 2018-01-01
FiscalYearEndDate 会計年度終了年月日 2018-12-31
会期: 19, 自の年月日:2018-01-01 00:00:00, 至の年月日: 2018-12-31 00:00:00
fiscal start 2018-01-01
fiscal end 2018-12-31


In [35]:
from lxml import etree
import re

def find_xpath_of_financial_statements(file_path):
    with open(file_path, 'rb') as file:
        xbrl_content = file.read()

    # Parse the content of the file
    tree = etree.XML(xbrl_content)
    document_tree = etree.ElementTree(tree)

    # Define a function to extract namespace map from the XML tree
    def get_namespace_map(xml_tree):
        nsmap = dict(xml_tree.nsmap)
        nsmap.pop(None, None)  # Remove the default namespace entry if it exists
        return {k: v for k, v in nsmap.items() if "link" not in k}

    ns_map = get_namespace_map(tree)

    # Titles of the financial statements with their corresponding keys
    titles = {
        "balance_sheet": "貸借対照表",
        "income_statement": "損益計算書",
        "cash_flow_statement": "キャッシュフロー計算書"
    }

    # Dictionary to store the XPaths
    xpaths = {}

    for key, title in titles.items():
        # Find elements that contain the title text more flexibly
        elements = tree.xpath(f"//*[contains(., '{title}')]", namespaces=ns_map)

        # If elements are found, store the XPath of the first occurrence
        if elements:
            # Getting the XPath of the first matching element
            xpaths[key] = document_tree.getpath(elements[0])
            # Printing some context around the found element
            print(f"Found '{title}' at: {xpaths[key]}")
            print("Context Data:", elements[0].text)
        else:
            xpaths[key] = None
            print(f"No elements found for '{title}'")

    return xpaths

# Specify the file path
file_path = '/content/jpcrp030000-asr-001_E31216-000_2023-06-30_01_2023-09-28.xbrl'

# Find the XPaths for the given financial statements
xpaths = find_xpath_of_financial_statements(file_path)

# Output the results
for statement, xpath in xpaths.items():
    print(f"XPath for {statement}: {xpath}")


FileNotFoundError: [Errno 2] No such file or directory: '/content/jpcrp030000-asr-001_E31216-000_2023-06-30_01_2023-09-28.xbrl'

In [ ]:
from bs4 import BeautifulSoup
import json

# Load the HTML content from a file
with open('/content/pl.html', 'r', encoding='utf-8') as file:
    html_content = file.read()

# Parse the HTML
soup = BeautifulSoup(html_content, 'lxml')

# Find the table containing the income statement
income_table = soup.find('table')  # Adjust the find method if there are multiple tables or specific identifiers

# Extract the data
income_data = []
rows = income_table.find_all('tr')
for row in rows:
    cells = row.find_all('td')
    if len(cells) == 3:  # Ensure there are exactly three columns as expected
        item = {
            "item": cells[0].get_text(strip=True),
            "previous_year": cells[1].get_text(strip=True),
            "current_year": cells[2].get_text(strip=True)
        }
        income_data.append(item)

# Exclude the details of cost of sales if necessary
income_data = [item for item in income_data if "売上原価明細" not in item["item"]]

# Convert the list of dictionaries into a JSON formatted string
json_data = json.dumps(income_data, ensure_ascii=False, indent=2)

print(json_data)


XBRLファイルダウンロード

In [ ]:
!pip install selenium
!wget https://chromedriver.storage.googleapis.com/114.0.5735.90/chromedriver_linux64.zip
!unzip /content/chromedriver_linux64.zip
!pip install webdriver-manager

In [ ]:
!chromedriver

In [ ]:
!PATH=$PATH:/content
!echo $PATH

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import time

# Chrome WebDriverのセットアップ
service = Service(ChromeDriverManager().install())
#service = Service('/content/chromedriver')
driver = webdriver.Chrome(service=service)

# ページにアクセス
driver.get('file:///content/書類簡易検索.htmll')  # ファイルパスを適切に設定してください

# JavaScriptがロードされるのを待つ
time.sleep(5)  # 必要に応じて調整

# XBRLリンクを見つける
xbrl_links = []
links = driver.find_elements_by_tag_name('a')
for link in links:
    if 'XBRL' in link.text:
        xbrl_links.append(link.get_attribute('href'))

# リンクを表示
print(xbrl_links)

# ブラウザを閉じる
driver.quit()


勘定科目一覧の作成

In [ ]:
import pandas as pd

# Excelファイルのパス
file_path = '/content/1f_AccountList.xlsx'

# Excelファイルの全シートを読み込む
xl = pd.ExcelFile(file_path)

# 各シートのデータをデータフレームとして辞書に格納
dfs = {sheet_name: xl.parse(sheet_name) for sheet_name in xl.sheet_names}

# '目次'と'勘定科目リストについて'という名前のシートは除外する
dfs.pop('目次', None)
dfs.pop('勘定科目リストについて', None)

print(dfs)


In [ ]:
import pandas as pd
import openpyxl

# Excelファイルのパス
file_path = '/content/1f_AccountList.xlsx'

# ファイルを読み込むための関数を定義
def load_excel(file_path, sheet_name=None):
    # シート名が指定されていない場合は、最初のシートを読み込む
    if sheet_name is None:
        sheet_name = openpyxl.load_workbook(file_path, read_only=True).sheetnames[0]
    # pandasを使ってExcelファイルのシートを読み込む
    return pd.read_excel(file_path, sheet_name=sheet_name)

# シート名を取得
wb = openpyxl.load_workbook(file_path, read_only=True)
sheet_names = wb.sheetnames
# '目次'と'勘定科目リストについて'という名前のシートは除外する
sheet_names.remove('目次')
sheet_names.remove('勘定科目リストについて')

ret1 = {}
ret2 = {}
# シートごとに処理
for sheet_name in sheet_names:
    df = load_excel(file_path, sheet_name)
    # ここで階層構造を解析する処理を記述する
    # 例えば、'depth'列が存在する場合、階層レベルに基づいて処理を行う
    #print(f"Processed sheet: {sheet_name}",df['Unnamed: 1'].tolist())
    ret1[sheet_name] = df['Unnamed: 1'].tolist()
    ret2[sheet_name] = df['Unnamed: 2'].tolist()

ret1
ret2

In [ ]:
ret1['一般商工業']
